In [1]:
# -*- coding: utf-8 -*-

import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re


In [4]:

train_data = pd.read_csv('train_.csv')
test_data = pd.read_csv('test_.csv')
data = train_data.append(test_data, ignore_index=True)
data['tweet'] = data['tweet'].str.replace("[^a-zA-Z#]", " ")

In [5]:
#removing the stop words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

data['tweet'] = data['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

tokenized = data['tweet'].apply(lambda x: x.split())

In [6]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized = tokenized.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

for i in range(len(tokenized)):
    tokenized[i] = ' '.join(tokenized[i])

data['tweet'] = tokenized

In [7]:
def hastags_collect(x):
    
    hashtags = []
    
    for i in x:
        ht = re.findall(r"#(\w+)",i)
        hashtags.append(ht)
        
    return hashtags

In [8]:
ht_positive = hastags_collect(data['tweet'][data['label']==0])

ht_negative = hastags_collect(data['tweet'][data['label']==1])

In [9]:
#from sklearn.feature_extraction.text import CountVectorizer
#bow_vec = CountVectorizer()
#bow = bow_vec.fit_transform(data['tweet'])
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(data['tweet'])

train_tfidf = tfidf[:7920,:]
test_tfidf = tfidf[7920:,:]

#from sklearn.model_selection import train_test_split
#xtrain_tfidf, xvalid_tfidf, ytrain, yvalid = train_test_split(train_tfidf, train_data['label'], random_state=42, test_size=0.3)
#
#xtrain_tfidf = train_tfidf[ytrain.index]
y = train_data['label']

In [10]:
#make a machine learning model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

reg = LogisticRegression()
reg.fit(train_tfidf, y)

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
dense_ = train_tfidf.toarray()
classifier.fit(dense_, y)

from sklearn.svm import SVC
classifier_svm = SVC(kernel = 'rbf', random_state = 0)
classifier_svm.fit(train_tfidf, y)

from sklearn.ensemble import RandomForestClassifier
random = RandomForestClassifier()
random.fit(train_tfidf,y)

from sklearn.naive_bayes import MultinomialNB
classifier_multi = MultinomialNB()
classifier_multi.fit(train_tfidf, y)

from sklearn.ensemble import GradientBoostingClassifier
classifier_multi = GradientBoostingClassifier()
classifier_multi.fit(train_tfidf,y)
#from sklearn.feature_extraction.text import CountVectorizer
#bow_vec = CountVectorizer()
#bow_test = bow_vec.fit_transform(test_data['tweet'])
#from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf_vectorizer_test = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
## TF-IDF feature matrix
#tfidf_test = tfidf_vectorizer_test.fit_transform(test_data['tweet'])


C:\Users\dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [12]:
prediction = reg.predict(test_tfidf)
prediction = reg.predict_proba(test_tfidf) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)



In [14]:
pred_naive = classifier.predict(dense_)

pred_svm = classifier_svm.predict(test_tfidf)

pred_rf = random.predict(test_tfidf)

pred_multinomial = classifier_multi.predict(test_tfidf)
#prediction_int_multi = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
#prediction_int_multi = prediction_int.astype(np.int)

#pred_GB = gb.predict(test_tfidf)
#pred_multinomial_GB = classifier_multi.predict(test_tfidf)